# AutoML in pipeline

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create a pipeline with Classification AutoML task.
- Create a pipeline with Regression AutoML task.

**Motivations** - This notebook explains how to use Classification & Regression AutoML task inside pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ml import MLClient, dsl, Input, command, Output
from azure.ml.automl import classification, regression
from azure.ml.entities._job.automl.tabular import TabularFeaturizationSettings

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

# 2. Basic pipeline job with Classification task

## 2.1 Build pipeline

In [ ]:
# note that the used docker image doesn't suit for all size of gpu compute. Please use the following command to create gpu compute if experiment failed
# !az ml compute create -n gpu-cluster --type amlcompute --min-instances 0 --max-instances 4 --size Standard_NC12

In [ ]:
# Define pipeline
@dsl.pipeline(
    description="AutoML Clasiification Pipeline",
    default_compute="cpu-cluster",
)
def automl_classification(
    classification_train_data,
    classification_validation_data
):
    # define the automl classification task with automl function
    classification_node = classification(
        training_data=classification_train_data,
        validation_data=classification_validation_data,
        target_column_name="y",
        primary_metric="accuracy",
        featurization=TabularFeaturizationSettings(mode="Auto"),
        # currently need to specify outputs "mlflow_model" explictly to reference it in following nodes 
        outputs={"best_model": Output(type="mlflow_model")},
    )
    # set limits and training
    classification_node.set_limits(max_trials=1)
    classification_node.set_training(enable_stack_ensemble=False, enable_vote_ensemble=False)

    command_func = command(
        inputs=dict(
            automl_output=Input(type="mlflow_model")
        ),
        command="ls ${{inputs.automl_output}}",
        environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1"
    )
    show_output = command_func(automl_output=classification_node.outputs.best_model)


data_folder = "../../../../cli/jobs/automl-standalone-jobs/cli-automl-classification-task-bankmarketing"
pipeline = automl_classification(
    classification_train_data=Input(path=f"{data_folder}/training-mltable-folder/", type="mltable"),
    classification_validation_data=Input(path=f"{data_folder}/validation-mltable-folder/", type="mltable"),
)

# 2.2 Submit pipeline job

In [ ]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline, experiment_name="pipeline_samples"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# 3. Basic pipeline job with regression task

## 3.1 Build pipeline

In [ ]:
# Define pipeline
@dsl.pipeline(
    description="AutoML Regression Pipeline",
    default_compute="cpu-cluster",
)
def automl_regression(
    regression_train_data,
    regression_validation_data,
    regression_test_data
):
    # define the AutoML regression task with AutoML function
    regression_node = regression(
        primary_metric="r2_score",
        target_column_name="SalePrice",
        training_data=regression_train_data,
        validation_data=regression_validation_data,
        test_data=regression_test_data,
        featurization=TabularFeaturizationSettings(mode="off"),
        # currently need to specify outputs "mlflow_model" explicitly to reference it in following nodes 
        outputs={"best_model": Output(type="mlflow_model")},
    )
    # set limits & training
    regression_node.set_limits(max_trials=1, max_concurrent_trials=1)
    regression_node.set_training(enable_stack_ensemble=False, enable_vote_ensemble=False)

    command_func = command(
        inputs=dict(
            automl_output=Input(type="mlflow_model")
        ),
        command="ls ${{inputs.automl_output}}",
        environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1"
    )
    show_output = command_func(automl_output=regression_node.outputs.best_model)


data_folder = "../../../../cli/jobs/pipelines/automl/house_pricing"
pipeline = automl_regression(
    regression_train_data=Input(path=f"{data_folder}/training-mltable-folder/", type="mltable"),
    regression_validation_data=Input(path=f"{data_folder}/validation-mltable-folder/", type="mltable"),
    regression_test_data=Input(path=f"{data_folder}/test-mltable-folder/", type="mltable")
)

# 3.2 Submit pipeline job

In [ ]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline, experiment_name="pipeline_samples"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../)